In [8]:
import cv2
import numpy as np
import pickle
from skimage.feature import local_binary_pattern, hog

In [15]:
# Parameters for LBP
radius = 2
n_points = 8 * radius
face_size = (128, 128)

In [16]:
# Load the trained model and label encoder
model_path = 'model/face_recognition_model.pkl'
label_encoder_path = 'model/label_encoder.pkl'
face_cascade_path = 'data/haarcascade_frontalface_default.xml'

with open(model_path, 'rb') as f:
    model = pickle.load(f)
with open(label_encoder_path, 'rb') as f:
    label_encoder = pickle.load(f)


In [17]:
# Initialize face cascade
face_cascade = cv2.CascadeClassifier(face_cascade_path)

In [18]:
# Feature extraction function using HOG and LBP
def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # HOG Features
    hog_features, _ = hog(gray, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True, feature_vector=True)
    
    # LBP Features
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype('float')
    lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalize histogram
    
    return np.hstack((hog_features, lbp_hist))

In [24]:

# load image path
image_path = 'Original Images/Original Images/Alia Bhatt/Alia Bhatt_15.jpg'  # Update with the path to your image


# Read and preprocess the image
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

for (x, y, w, h) in faces:
    face = image[y:y+h, x:x+w]
    face = cv2.resize(face, face_size)
    features = extract_features(face)
    
    # Predict the label
    label = model.predict([features])[0]
    label_name = label_encoder.inverse_transform([label])[0]
    
    # Draw the label on the image
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(image, label_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image with label
cv2.imshow('Image with Label', image)
cv2.waitKey(0)
cv2.destroyAllWindows()